In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
x = torch.tensor([
    [0.,0],
    [0,1],
    [1,1]
])
print(x)
print(x[None,:,:])

tensor([[0., 0.],
        [0., 1.],
        [1., 1.]])
tensor([[[0., 0.],
         [0., 1.],
         [1., 1.]]])


In [3]:
def method1(x):
    xx = F.pdist(x)
    m = torch.zeros((x.shape[0],x.shape[0]))
    triu_indices = torch.triu_indices(row=x.shape[0], col=x.shape[0], offset=1)
    m[triu_indices[0], triu_indices[1]] = xx
    m[triu_indices[1], triu_indices[0]] = xx
    return m


def method2(x):
    return torch.norm(x[:, None] - x, dim=2, p=2)


def method3(x):
    return (x[:, :, None, :] - x[:, None, :, :]).norm(p=2, dim=-1)

In [4]:
x = torch.rand((100,2))

In [5]:
%timeit method1(x)


105 µs ± 406 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%timeit method2(x)

1.74 ms ± 2.16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit method3(x[None,:,:])

1.75 ms ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
print(torch.norm(method1(x)[:,:] - method2(x)[:,:]))
print(torch.norm(method1(x)[:,:] - method3(x[None,:,:])[:,:]))
print(torch.norm(method2(x)[:,:] - method3(x[None,:,:])[:,:]))

tensor(0.)
tensor(0.)
tensor(0.)


In [9]:
model = nn.Embedding(100,128)

print(sum(t.numel() for t in model.parameters() if t.requires_grad))

model = nn.Linear(100,128)

print(sum(t.numel() for t in model.parameters() if t.requires_grad))

12800
12928


In [10]:
pi = torch.tensor([
    [0,2,1],
    [2,1,0]
])
# dataset = torch.tensor([
#     [
#         [1,1.],
#         [2,2],
#         [3,3]
#     ],
#     [
#         [-1,-1],
#         [-2,-2],
#         [-3,-3]
#     ]
# ])

for _ in range(10000):
    dataset = torch.rand((2,3,2))
    d = dataset.gather(1, pi.unsqueeze(-1).expand_as(dataset))
    res = (d[:, 1:] - d[:, :-1]).norm(p=2, dim=2).sum(1) + (d[:, 0] - d[:, -1]).norm(p=2, dim=1)

    # metodo con distanze
    dataset_dist = torch.stack(tuple(method1(d) for d in dataset),0)
    a = torch.arange(pi.shape[1])
    idx = torch.stack((a, a.roll(-1,0)))
    res_dist = dataset_dist[:,idx[0],idx[1]].sum(1)

    assert torch.norm(res - res_dist) <= 1e-6, f'{res} {res_dist}'

In [11]:
x = torch.tensor([
    [[1,1],[2,2],[3,3],[4,4]],
    [[5,5],[6,6],[7,7],[8,8]]])
print(x[:,None,:,:].shape,x[:,None,:,:])
print(x[:,:,None,:].shape, x[:,:,None,:])

torch.Size([2, 1, 4, 2]) tensor([[[[1, 1],
          [2, 2],
          [3, 3],
          [4, 4]]],


        [[[5, 5],
          [6, 6],
          [7, 7],
          [8, 8]]]])
torch.Size([2, 4, 1, 2]) tensor([[[[1, 1]],

         [[2, 2]],

         [[3, 3]],

         [[4, 4]]],


        [[[5, 5]],

         [[6, 6]],

         [[7, 7]],

         [[8, 8]]]])


In [12]:
dataset = torch.stack([torch.FloatTensor(10,2).uniform_(0,1) for i in range(20)])
print(dataset.shape)

torch.Size([20, 10, 2])


In [13]:
%timeit torch.linalg.norm((dataset[:,:,None,:] - dataset[:,None,:,:]),dim=-1)

371 µs ± 305 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%timeit (dataset[:,:,None,:] - dataset[:,None,:,:]).norm(p=2,dim=-1)

374 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
tmp = (dataset[:,:,None,:] - dataset[:,None,:,:])
i = torch.ones(tmp.shape)
i[:,:,:,-2:] = 2
z = torch.linalg.norm((dataset[:,:,None,:] - dataset[:,None,:,:]),dim=-1)
w = (dataset[:,:,None,:] - dataset[:,None,:,:]).norm(p=2,dim=-1)
print((z - w).norm())
print(z)
print(w)

tensor(0.)
tensor([[[0.0000, 0.2798, 0.3240,  ..., 0.7086, 0.2217, 0.3238],
         [0.2798, 0.0000, 0.3700,  ..., 0.7855, 0.3841, 0.1316],
         [0.3240, 0.3700, 0.0000,  ..., 1.0296, 0.1517, 0.2810],
         ...,
         [0.7086, 0.7855, 1.0296,  ..., 0.0000, 0.9248, 0.9110],
         [0.2217, 0.3841, 0.1517,  ..., 0.9248, 0.0000, 0.3438],
         [0.3238, 0.1316, 0.2810,  ..., 0.9110, 0.3438, 0.0000]],

        [[0.0000, 0.1014, 0.4744,  ..., 0.4779, 0.5150, 0.1857],
         [0.1014, 0.0000, 0.5662,  ..., 0.5792, 0.4209, 0.2236],
         [0.4744, 0.5662, 0.0000,  ..., 0.2518, 0.9871, 0.3963],
         ...,
         [0.4779, 0.5792, 0.2518,  ..., 0.0000, 0.9731, 0.4977],
         [0.5150, 0.4209, 0.9871,  ..., 0.9731, 0.0000, 0.6236],
         [0.1857, 0.2236, 0.3963,  ..., 0.4977, 0.6236, 0.0000]],

        [[0.0000, 0.0685, 0.1839,  ..., 0.2129, 0.5792, 0.3699],
         [0.0685, 0.0000, 0.1351,  ..., 0.2120, 0.5118, 0.3143],
         [0.1839, 0.1351, 0.0000,  ..., 0.3375,

In [31]:
dataset = torch.stack([torch.FloatTensor(100,2).uniform_(0,1) for i in range(512)])
print(dataset.shape)
tmp = (dataset[:,:,None,:] - dataset[:,None,:,:]).norm(p=2,dim=-1).sort(-1)[0][:,:,1:]
dataset = torch.cat((dataset,tmp),dim=-1)
print(dataset.shape)

print(dataset[:10,:10,:10])

torch.Size([512, 100, 2])
torch.Size([512, 100, 101])
tensor([[[0.3351, 0.5061, 0.1254, 0.1397, 0.1421, 0.1568, 0.1809, 0.2160,
          0.2162, 0.2182],
         [0.6541, 0.8880, 0.0913, 0.1107, 0.1670, 0.1687, 0.1705, 0.1870,
          0.1931, 0.2004],
         [0.5576, 0.4276, 0.0712, 0.0789, 0.0818, 0.1007, 0.1011, 0.1234,
          0.1340, 0.1382],
         [0.4152, 0.7553, 0.0514, 0.0943, 0.1236, 0.1512, 0.1515, 0.1623,
          0.1631, 0.1655],
         [0.7283, 0.5666, 0.1108, 0.1313, 0.1445, 0.1469, 0.1489, 0.1562,
          0.1602, 0.1698],
         [0.5849, 0.7142, 0.0247, 0.0338, 0.0560, 0.1150, 0.1234, 0.1747,
          0.1819, 0.1870],
         [0.3322, 0.9672, 0.0245, 0.1078, 0.1298, 0.1370, 0.1531, 0.1656,
          0.2134, 0.2275],
         [0.2817, 0.8477, 0.0873, 0.1059, 0.1123, 0.1241, 0.1298, 0.1492,
          0.1515, 0.1623],
         [0.8532, 0.6816, 0.0390, 0.0487, 0.0658, 0.0702, 0.0708, 0.0956,
          0.1081, 0.1214],
         [0.9648, 0.9102, 0.0849, 0.1

In [36]:
a = torch.tensor([1,2,3])
print(torch.cat((a[:1],torch.tensor([4,5,6]))))

tensor([1, 4, 5, 6])
